In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                          trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                             trust_remote_code=True).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
examples = [
    'she likes green.',
    'it is raining.'
]

question = tokenizer.encode(
    'what is the weather?<|end|>\n<|assistant|>',
    return_tensors='pt'
)[0][1:].to(device)

examples_tokens = [
    tokenizer.encode(
        f'<|user|>\n{example}\n',
        return_tensors='pt'
    )[0].to(device) for example in examples
]

In [ ]:
examples_embeddings = torch.stack([
    model.get_input_embeddings()(example_tokens)
    for example_tokens in examples_tokens
], dim=0).mean(dim=0)

In [ ]:
tokens = question

for _ in range(30):
  input_embeddings = model.get_input_embeddings()(tokens)
  next_token = model(
      inputs_embeds=torch.cat([examples_embeddings, input_embeddings])[None]
  ).logits[:, -1, :].argmax(dim=-1)
  tokens = torch.cat([tokens, next_token])

print(tokenizer.decode(tokens))

what is the weather?<|end|><|assistant|> The statement "it is green" is not typically used to describe weather. However, if you're referring to the color of the sky or fol


In [ ]:
mean_past_key_values = [
    [
        torch.stack([
            model(example_tokens[None]).past_key_values[layer_index][kv]
            for example_tokens in examples_tokens
        ], dim=0).mean(dim=0)
        for kv in range(2)
    ]
    for layer_index in range(model.config.num_hidden_layers)
]

In [ ]:
tokens = question

for i in range(30):
  next_token = model(
      tokens[None],
      past_key_values=mean_past_key_values
  ).logits[:, -1, :].argmax(dim=-1)
  tokens = torch.cat([tokens, next_token])

print(tokenizer.decode(tokens))

what is the weather?<|end|><|assistant|> The statement "it is raining" could be a reasonable inference from the information that "she likes it when it is raining," but it
